In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.feature_selection import RFE
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # ✅ 用于进度显示
from imblearn.over_sampling import SMOTE  # 导入 SMOTE
import warnings
warnings.filterwarnings("ignore")
import os
import glob

def read_single_csv(input_path):
    total_rows = sum(1 for _ in open(input_path, encoding='utf-8')) - 1  # 估算总行数（除去 header）
    chunks = pd.read_csv(input_path, chunksize=10000)
    df_list = []
    for chunk in tqdm(chunks, total=total_rows // 10000 + 1, desc=f"Reading {os.path.basename(input_path)}"):
        df_list.append(chunk)
    return pd.concat(df_list, ignore_index=True)


feature_start = 10
feature_num = 50
folder_path = 'D:\\trainSet'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))  # 读取该目录下所有 CSV 文件

# --------------------------
# 多文件数据读取
all_dataframes = []
for file in csv_files:
    df = read_single_csv(file)
    all_dataframes.append(df)

data = pd.concat(all_dataframes, ignore_index=True)


# 缺失值处理
for col in ['feature21', 'feature22', 'feature23']:
    data[col] = data[col].fillna(data[col].mean())

data[['feature46', 'feature47', 'feature48']] = data[['feature46', 'feature47', 'feature48']].fillna(0)
data['feature48'] = data['feature48'].replace([np.inf, -np.inf], 0)

print(f"before_dropna_data.shape: {data.shape}")
data = data.dropna().reset_index(drop=True)
print(f"after_dropna_data.shape: {data.shape}")



def split_decoy(data_chunk):
    is_decoy = data_chunk.iloc[:, 4].str.contains('DECOY', na=False)
    return data_chunk[~is_decoy], data_chunk[is_decoy]

chunks = [data.iloc[i:i+1000] for i in range(0, len(data), 1000)]
data_sp_list = []
data_decoy_list = []

with ThreadPoolExecutor(max_workers=30) as executor:
    futures = {executor.submit(split_decoy, chunk): i for i, chunk in enumerate(chunks)}
    for future in tqdm(as_completed(futures), total=len(chunks), desc="Splitting decoys"):
        sp_part, decoy_part = future.result()
        data_sp_list.append(sp_part)
        data_decoy_list.append(decoy_part)



data_sp = pd.concat(data_sp_list, ignore_index=True)
data_decoy = pd.concat(data_decoy_list, ignore_index=True)



max_decoy_match_peak = data_decoy['match peak'].max()
min_th = int(max_decoy_match_peak * 3 / 10)
min_th = max(min_th, 5)

data = data[(data['match peak'] <= max_decoy_match_peak) & (data['match peak'] >= min_th)].reset_index(drop=True)

columns_to_drop = []

# 删除这些列
data = data.drop(columns=columns_to_drop, errors='ignore')

# 特征提取
feature = data.iloc[:, feature_start:feature_start + feature_num - len(columns_to_drop)]
protein = data.iloc[:, 4]
target = np.zeros((len(protein), 1))

# 输出信息
print(f"data_decoy.shape: {data_decoy.shape}")
print(f"data_sp.shape: {data_sp.shape}")

# 显示统计信息
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
print(data.describe())


Reading Yeast7out_all_prsm.csv: 100%|████████████████████████████████████████████████| 654/654 [01:15<00:00,  8.70it/s]


before_dropna_data.shape: (21361847, 60)
after_dropna_data.shape: (21361847, 60)


Splitting decoys: 100%|████████████████████████████████████████████████████████| 21362/21362 [00:02<00:00, 7183.28it/s]


data_decoy.shape: (13695551, 60)
data_sp.shape: (7666296, 60)
        Spectrum ID       Scan(s)  Precursor mass  Adjusted precursor mass  \
count  1.508912e+06  1.508912e+06    1.508912e+06             1.508912e+06   
mean   9.765237e+02  3.560600e+03    1.028381e+04             1.028380e+04   
std    4.524535e+02  1.429600e+03    4.406037e+03             4.406043e+03   
min    0.000000e+00  2.000000e+00    1.460740e+03             1.460736e+03   
25%    6.830000e+02  2.618000e+03    7.254423e+03             7.254394e+03   
50%    9.340000e+02  3.212000e+03    9.445721e+03             9.445821e+03   
75%    1.206000e+03  4.215000e+03    1.255158e+04             1.255156e+04   
max    3.974000e+03  1.237200e+04    3.275887e+04             3.276029e+04   

       First residue  Last residue    match peak  match fragment  \
count   1.508912e+06  1.508912e+06  1.508912e+06    1.508912e+06   
mean    1.745804e+02  2.679189e+02  1.746431e+01    1.329201e+01   
std     4.174034e+02  4.188568e

In [3]:
for i in range(0, len(protein)):
    if "DECOY" in str(protein[i]):
        target[i] = 0
    else:
        target[i] = 1

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.25, random_state=77)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=77)

print("训练集：", x_train.shape, y_train.shape)
print("验证集：", x_val.shape, y_val.shape)
print("测试集：", x_test.shape, y_test.shape)

# SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=77)  # 'auto' 是默认值，表示平衡样本数量
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

print(f"SMOTE: {x_train_resampled.shape}, {y_train_resampled.shape}")


base_clf = RandomForestClassifier(
    n_estimators=1500,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    n_jobs=30,
    verbose=1,
    random_state=77
)


n_features_to_select = 40
selector = RFE(estimator=base_clf, n_features_to_select=n_features_to_select, step=1, verbose=1)
selector.fit(x_train_resampled, y_train_resampled)


x_train_selected = selector.transform(x_train_resampled)
x_val_selected = selector.transform(x_val)
x_test_selected = selector.transform(x_test)

clf = RandomForestClassifier(
    n_estimators=1500,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    n_jobs=30,
    verbose=1,
    random_state=77
)
clf.fit(x_train_selected, y_train_resampled)


score_val = clf.score(x_val_selected, y_val)
print("在验证集上的得分：", score_val)
score_test = clf.score(x_test_selected, y_test)
print("在测试集上的得分：", score_test)

predict = clf.predict(x_test_selected)
print(predict)


print(classification_report(y_test, predict, labels=[0, 1], target_names=["decoy", "target"]))


importance = clf.feature_importances_
print("特征重要性：", importance)


selected_features = selector.get_support(indices=True)
print("被选择的特征索引：", selected_features)

训练集： (848763, 50) (848763, 1)
验证集： (282921, 50) (282921, 1)
测试集： (377228, 50) (377228, 1)
SMOTE 后训练集的大小: (947260, 50), (947260,)
Fitting estimator with 50 features.


[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:  2.2min
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:  5.0min
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:  9.1min
[Parallel(n_jobs=30)]: Done 1190 tasks      | elapsed: 14.1min
[Parallel(n_jobs=30)]: Done 1500 out of 1500 | elapsed: 17.6min finished


Fitting estimator with 49 features.


[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:  1.9min
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:  4.9min
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:  9.0min
[Parallel(n_jobs=30)]: Done 1190 tasks      | elapsed: 14.3min
[Parallel(n_jobs=30)]: Done 1500 out of 1500 | elapsed: 17.9min finished


Fitting estimator with 48 features.


...

在验证集上的得分： 0.6565790450337726


[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:    0.9s
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:    2.3s
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:    4.3s
[Parallel(n_jobs=30)]: Done 1190 tasks      | elapsed:    6.8s
[Parallel(n_jobs=30)]: Done 1500 out of 1500 | elapsed:    8.6s finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.


在测试集上的得分： 0.6550600697721272


[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:    0.8s
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:    2.2s
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:    4.2s
[Parallel(n_jobs=30)]: Done 1190 tasks      | elapsed:    6.6s
[Parallel(n_jobs=30)]: Done 1500 out of 1500 | elapsed:    8.2s finished


[0. 0. 0. ... 0. 0. 0.]
              precision    recall  f1-score   support

       decoy       0.63      0.94      0.75    210852
      target       0.80      0.29      0.43    166376

    accuracy                           0.66    377228
   macro avg       0.71      0.62      0.59    377228
weighted avg       0.70      0.66      0.61    377228

特征重要性： [0.01481816 0.01685946 0.0437489  0.02237148 0.01633872 0.01694838
 0.01870572 0.01944728 0.03340064 0.03207457 0.05087286 0.03241461
 0.04480285 0.01446338 0.01621905 0.01841945 0.03296061 0.01521056
 0.01561815 0.0175869  0.01741153 0.0227243  0.01495716 0.01501942
 0.01409073 0.01727884 0.02429922 0.02895415 0.06397317 0.01421842
 0.0395901  0.02261906 0.02377124 0.0254177  0.03434424 0.01545549
 0.01600256 0.01380532 0.05088638 0.03189921]
被选择的特征索引： [ 0  1  2  5  6  7  8  9 12 13 14 15 16 19 20 21 22 24 25 26 27 28 29 30
 31 33 34 35 36 37 38 39 40 41 42 44 46 47 48 49]


In [4]:
import joblib # 新版本 Scikit-learn
joblib.dump(clf, "D:\\RFModel\\rf_model_513.m")
joblib.dump(selector, 'D:\\RFModel\\rf_model_513_selector.pkl')

['D:\\LYC\\WTop2025_4_3\\RFModel\\rf_model_513.m']

In [6]:
data_sp = data_sp[(data_sp['match peak'] <= max_decoy_match_peak) & (data_sp['match peak'] >= min_th)].reset_index(drop=True)
print(data_sp.describe())

         Spectrum ID        Scan(s)  Precursor mass  Adjusted precursor mass  \
count  666491.000000  666491.000000   666491.000000            666491.000000   
mean      988.908937    3566.851093    10074.197840             10074.181390   
std       461.041472    1457.116037     4478.694919              4478.699185   
min         0.000000       2.000000     1460.739700              1460.735900   
25%       685.000000    2615.000000     6979.395700              6979.405700   
50%       954.000000    3209.000000     9183.949100              9183.911000   
75%      1213.000000    4212.000000    12285.498100             12285.528100   
max      3974.000000   12372.000000    32758.870800             32760.293900   

       First residue   Last residue     match peak  match fragment  \
count  666491.000000  666491.000000  666491.000000   666491.000000   
mean      163.580797     255.162634      18.098028       13.834377   
std       336.203792     337.994357       5.112685        4.488173   

In [7]:
data_decoy = data_decoy[(data_decoy['match peak'] <= max_decoy_match_peak) & (data_decoy['match peak'] >= min_th)].reset_index(drop=True)
print(data_decoy.describe())

         Spectrum ID        Scan(s)  Precursor mass  Adjusted precursor mass  \
count  842421.000000  842421.000000   842421.000000            842421.000000   
mean      966.725034    3555.655187    10449.655999             10449.638471   
std       445.297972    1407.431096     4340.528983              4340.535749   
min         0.000000       2.000000     1824.168600              1824.158600   
25%       682.000000    2618.000000     7293.480100              7293.475000   
50%       914.000000    3213.000000     9597.958100              9597.888100   
75%      1200.000000    4215.000000    12771.749200             12771.736100   
max      3974.000000   12372.000000    32758.870800             32759.010800   

       First residue   Last residue     match peak  match fragment  \
count  842421.000000  842421.000000  842421.000000   842421.000000   
mean      183.282865     278.011081      16.962935       12.862912   
std       471.664703     472.895744       3.243900        3.056715   